In [ ]:
import pandas as pd
import sys, os, gc
sys.path.append("/home/fvalle/phd/master_thesis/hsbm/")
sys.path.append("/home/fvalle/phd/master_thesis/")
os.chdir("/home/fvalle/phd/datasets/unbalanced/")

In [ ]:
#GTEx
df = pd.read_csv('https://storage.googleapis.com/gtex_analysis_v8/rna_seq_data/GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_tpm.gct.gz', skiprows=2, compression='gzip', sep='\t')
df['ensg'] = [x[:15] for x in df['Name']]
df.set_index('Name', inplace=True)
df.set_index(['ensg'],inplace=True)
df=df.drop(['Description'],1)
genelist=pd.read_csv("https://stephenslab.github.io/count-clustering/project/utilities/gene_names_all_gtex.txt", header=None).values.ravel()
df = df[df.index.isin(genelist)]
df_file = pd.read_csv("https://storage.googleapis.com/gtex_analysis_v8/annotations/GTEx_Analysis_v8_Annotations_SampleAttributesDS.txt", sep='\t').loc[:,['SAMPID','SMTS', 'SMTSD']]
df_file.set_index('SAMPID', inplace=True)

In [ ]:
df_file.groupby('SMTS').count().sort_values('SMTSD',ascending=False)

In [ ]:
df_ub = df.reindex(columns=df_file[df_file['SMTS'].isin(['Stomach', 'Uterus', 'Kidney'])].index).dropna(how='all', axis=1)

In [ ]:
import scanpy as sc

In [ ]:
adata = sc.AnnData(X=df_ub.transpose(), obs=df_file.reindex(index=df_ub.columns))
sc.pp.log1p(adata, copy=False)
sc.pp.highly_variable_genes(adata, min_mean=0, max_mean=10, n_bins=40)
sc.pl.highly_variable_genes(adata, save="hvg.svg")

In [ ]:
hvg = adata.var[adata.var['highly_variable']==True].sort_values(by='dispersions_norm', ascending=False).index
samples = adata.obs.index

In [ ]:
df_hsbm = df.reindex(index=hvg[:1000], columns = samples).applymap(lambda x: pd.np.log2(x+1)*1e2)
df_hsbm

In [ ]:
df_hsbm.to_csv("mainTable.csv", index=True, header=True)
df_file[df_file.index.isin(df_hsbm.columns)].to_csv("files.dat", index=True, header=True)

In [ ]:
gc.collect()

In [ ]:
from sbmtm import sbmtm

In [ ]:
if model:
    del model
    model = sbmtm()
    gc.collect()

In [ ]:
model.load_graph("graph.xml.gz")

In [ ]:
model.make_graph_from_BoW_df(df_hsbm)
model.save_graph("graph.xml.gz")

In [ ]:
model.fit(verbose=True, parallel=True)

In [ ]:
model.save_data()

In [ ]:
from hsbmpy import clusteranalysis
from hsbmpy import get_max_available_L

In [ ]:
clusteranalysis(os.getcwd(),labels=['SMTS'], l_max=get_max_available_L(os.getcwd()))